In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten,Reshape
from keras.models import Model,load_model
from keras import backend as K
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [0]:
input_img = Input(shape=(50, 50, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)

encoded = Dense(32,activation='relu')(x)
# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Dense(64,activation='relu')(encoded)
x = Reshape([4,4,4])(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='valid')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='valid')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='valid')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [0]:
autoencoder.summary()

In [0]:
!unzip dataset2.zip

In [0]:
from keras.preprocessing.image import ImageDataGenerator
directory = '/content/dataset2'

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    rotation_range=10,
    width_shift_range=5.0,
    height_shift_range=5.0,
    brightness_range=None)

train_generator = train_datagen.flow_from_directory(
    directory,
    target_size=(50, 50),
    batch_size=1328,color_mode='grayscale')

In [0]:
for i in train_generator:
  print(i[0].shape)
  break
cv2_imshow(i[0][0,:,:,0]*255)

In [0]:
epochs =500
for epoch in range(epochs):
  print('EPOCH: ',epoch)
  for i in train_generator:
    X = i[0]
    autoencoder.fit(X,X,epochs=1,batch_size=4)
    break
  if epoch%20==0:
    n =10
    pred= autoencoder.predict(X[:n])
    plt.figure(figsize=(17,7))
    for i in range(n):
      plt.subplot(2,n,i+1)
      plt.imshow(X[i,0:,:,0])
      plt.subplot(2,n,i+n+1)
      plt.imshow(pred[i,0:,:,0])
    plt.show()
  if epoch%50==0:
    autoencoder.save('/content/drive/My Drive/Colab_Notebooks/Handwriting/autoencoder.model')

In [0]:
autoencoder = load_model('/content/drive/My Drive/Colab_Notebooks/Handwriting/autoencoder.model')

Separate encoder decoder model from autoencoder

In [0]:
input_layer_encode = Input(shape=(50, 50, 1))
for i in range(1,9):
  if i==1:
    x = autoencoder.layers[i](input_layer_encode)
  else:
    x = autoencoder.layers[i](x)
encoder = Model(input_layer_encode,x)

input_layer_dencode = Input(shape=(32,))
for i in range(9,19):
  if i==9:
    x = autoencoder.layers[i](input_layer_dencode)
  else:
    x = autoencoder.layers[i](x)
dencoder = Model(input_layer_dencode,x)



In [0]:
text = list('0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ(){}[]+-=:%')+['slash']
vectors = {i:[] for i in text}
for folder in os.listdir('/content/dataset2/'):
  for img in os.listdir('/content/dataset2/'+folder):
    #print(img)
    image= cv2.imread('/content/dataset2/'+folder+'/'+img,0)
    #print(image.shape)
    cv2_imshow(image)
    cv2_imshow(autoencoder.predict(image.reshape(1,50,50,1)/255)[0,:,:,0]*255)
    pred = encoder.predict(image.reshape(1,50,50,1)/255)

    vectors[folder].append(pred)

    print(pred)
    for i in range(32):
      if int(pred[0][i])!=0: 
        pred[0][i]+=np.random.uniform(low=0.1,high=0.5,size=1)[0]
    cv2_imshow(dencoder.predict(pred)[0,:,:,0]*255)
    print('\n\n\n')

In [0]:
import pickle
with open('/content/drive/My Drive/Colab_Notebooks/Handwriting/vectors.pickle', 'wb') as f:
    pickle.dump(vectors, f)

Test the decoder for generating characters

In [0]:
def get_new(array):
  l= len(array)
  s = np.random.randint(low=0,high=l,size=1)[0]
  print(s)
  new_pred = array[s]
  for i in range(32):
    new_pred[0][i]+=np.random.uniform(low=0,high=0.3,size=1)[0]
  return np.array(new_pred).reshape(1,-1)

In [0]:
new = get_new(vectors['A'])
cv2_imshow(dencoder.predict(new)[0,:,:,0]*255)

In [0]:
dencoder.save('/content/drive/My Drive/Colab_Notebooks/Handwriting/decoder.model')